Importing Libraries

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.svm import SVC
import random


Loading the Cleveland Heart Disease Dataset

In [3]:
# Load the Cleveland Heart Disease dataset
cleveland_data = pd.read_csv('Heart_disease_cleveland_new.csv', names=[
    "age", "sex", "cp", "trestbps", "chol", "fbs", "restecg", "thalach", "exang", "oldpeak",
    "slope", "ca", "thal", "target"
], na_values='?')

# Load the Statlog (Heart) dataset
statlog_url = "https://archive.ics.uci.edu/ml/machine-learning-databases/statlog/heart/heart.dat"
statlog_column_names = [
    "age", "sex", "cp", "trestbps", "chol", "fbs", "restecg", "thalach", "exang", "oldpeak",
    "slope", "ca", "thal", "target"
]
statlog_data = pd.read_csv(statlog_url, names=statlog_column_names, delim_whitespace=True)
# Adjust 'slope' and 'thal' values in Statlog dataset to match Cleveland dataset
statlog_data['slope'] = statlog_data['slope'].replace({1: 1, 2: 2, 3: 0})
statlog_data['thal'] = statlog_data['thal'].replace({3: 1, 6: 2, 7: 3})
statlog_data['target'] = statlog_data['target'].replace({1: 0, 2: 1})
print(statlog_data.head())

# Combine the datasets
data = pd.concat([cleveland_data, statlog_data])
# Display the rows where the target value is 2
# target_2_data = cleveland_data[cleveland_data['target'] == 2]
# print(target_2_data.head())
# print(len(target_2_data.index))
# Display the first few rows of the combined dataset
print(data.head())
print(len(data.index))


/tmp/ipykernel_1508/1457845952.py:13: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  statlog_data = pd.read_csv(statlog_url, names=statlog_column_names, delim_whitespace=True)


    age  sex   cp  trestbps   chol  fbs  restecg  thalach  exang  oldpeak  \
0  70.0  1.0  4.0     130.0  322.0  0.0      2.0    109.0    0.0      2.4   
1  67.0  0.0  3.0     115.0  564.0  0.0      2.0    160.0    0.0      1.6   
2  57.0  1.0  2.0     124.0  261.0  0.0      0.0    141.0    0.0      0.3   
3  64.0  1.0  4.0     128.0  263.0  0.0      0.0    105.0    1.0      0.2   
4  74.0  0.0  2.0     120.0  269.0  0.0      2.0    121.0    1.0      0.2   

   slope   ca  thal  target  
0    2.0  3.0   1.0       1  
1    2.0  0.0   3.0       0  
2    1.0  0.0   3.0       1  
3    2.0  1.0   3.0       0  
4    1.0  1.0   1.0       0  
   age  sex  cp  trestbps  chol  fbs  restecg  thalach  exang  oldpeak  slope  \
0  age  sex  cp  trestbps  chol  fbs  restecg  thalach  exang  oldpeak  slope   
1   63    1   0       145   233    1        2      150      0      2.3      2   
2   67    1   3       160   286    0        2      108      1      1.5      1   
3   67    1   3       120   229  

Data Pre-Processing

In [8]:
data['target'] = pd.to_numeric(data['target'], errors='coerce')
# Handle missing values by dropping rows with missing values
data.dropna(inplace=True)

# Convert target column to integer type
data['target'] = data['target'].astype(int)

# Split the data into features and target
X = data.drop('target', axis=1)
y = data['target'].apply(lambda x: 1 if x > 0 else 0)  # Convert target to binary

# Normalize the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)


JellyFish Optimization Algorithm

In [9]:
class JellyfishOptimizationAlgorithm:
    def __init__(self, population_size, dimensions, lower_bound, upper_bound, max_iter):
        self.population_size = population_size
        self.dimensions = dimensions
        self.lower_bound = lower_bound
        self.upper_bound = upper_bound
        self.max_iter = max_iter
        self.population = self.initialize_population()

    def initialize_population(self):
        return np.random.uniform(self.lower_bound, self.upper_bound, (self.population_size, self.dimensions))

    def fitness(self, solution):
        # Ensure gamma is a positive value
        C = solution[0]
        gamma = abs(solution[1])  # Taking absolute value to ensure gamma is positive
        model = SVC(C=C, gamma=gamma, kernel='rbf')
        model.fit(X_train, y_train)
        predictions = model.predict(X_train)
        accuracy = accuracy_score(y_train, predictions)
        return accuracy

    def optimize(self):
        best_solution = None
        best_fitness = -1

        for iteration in range(self.max_iter):
            for individual in self.population:
                fitness_value = self.fitness(individual)
                if fitness_value > best_fitness:
                    best_fitness = fitness_value
                    best_solution = individual

            # Update population
            for i in range(self.population_size):
                if random.random() < 0.5:
                    self.population[i] = best_solution + np.random.uniform(-1, 1, self.dimensions)
                else:
                    self.population[i] = np.random.uniform(self.lower_bound, self.upper_bound, self.dimensions)

        return best_solution, best_fitness


Using SVM with JOA to predict heart diseases

In [11]:
# Define the bounds for C and gamma parameters of the SVM
lower_bound = [0.1, 0.001]
upper_bound = [100, 1]

# Instantiate the Jellyfish Optimization Algorithm
joa = JellyfishOptimizationAlgorithm(
    population_size=30, dimensions=2, lower_bound=lower_bound, upper_bound=upper_bound, max_iter=50
)

# Optimize the SVM parameters using JOA
best_solution, best_fitness = joa.optimize()
print(f"Best solution (C, gamma): {best_solution}, Best fitness: {best_fitness}")

# Train the SVM model with the best parameters
model = SVC(C=best_solution[0], gamma=best_solution[1], kernel='rbf')
model.fit(X_train, y_train)

# Evaluate the model
y_pred = model.predict(X_test)
print(f"Accuracy: {accuracy_score(y_test, y_pred)}")
print(f"Precision: {precision_score(y_test, y_pred)}")
print(f"Recall: {recall_score(y_test, y_pred)}")
print(f"F1 Score: {f1_score(y_test, y_pred)}")
print(f"ROC AUC Score: {roc_auc_score(y_test, y_pred)}")


Best solution (C, gamma): [8.67671283 1.07303811], Best fitness: 1.0
Accuracy: 0.8434782608695652
Precision: 0.9210526315789473
Recall: 0.7
F1 Score: 0.7954545454545454
ROC AUC Score: 0.8269230769230769
